In [419]:
import numpy as np
from pandas import Series, DataFrame
import pandas as pd
from pandas import read_html

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

In [420]:
#Constants for accessing DataFrame data pulled from web
QB = 0
RB = 1
WR = 2
TE = 3
DST = 4
K = 5

In [581]:
#Set to create unique in house DataFrame 

tupOffense = [('', 'Game'),        
              ('', 'Pts'),
              ('Passing', 'Att'),
              ('Passing', 'Cmp'),
              ('Passing', 'Yds'),
              ('Passing', 'TD'),
              ('Passing', 'Int'),
              ('Passing', '2Pt'),
              ('Rushing', 'Att'),
              ('Rushing', 'Yds'),
              ('Rushing', 'TD'),
              ('Rushing', '2Pt'),
              ('Receiving', 'Rec'),
              ('Receiving', 'Yds'),
              ('Receiving', 'TD'),
              ('Receiving', '2Pt'),
              ('Fumbles', 'FL'),
              ('Fumbles', 'TD')
             ]   

In [605]:
tupDef = [('', 'Opp'),
          ('', 'Pts*'),
          ('Def', 'Sack'), 
          ('Def', 'Int'),
          ('Def', 'Saf'), 
          ('Def', 'FR'), 
          ('Def', 'Blk'), 
          ('Def', 'TD'), 
          ('Def', 'PA'),
          ('Def', 'PassYds/G'), 
          ('Def', 'RushYds/G'), 
          ('Def', 'TotYds/G')
         ]

In [606]:
tupKicker = [('', 'Game'), 
             ('', 'Pts*'),          
             ('Kicking', 'XPA'),
             ('Kicking', 'XPM'),
             ('Kicking', 'FGA'),
             ('Kicking', 'FGM'),
             ('Kicking', '50+'),
            ]

In [424]:
#Set to create unique in house DataFrame of master Dataframe that will have all players and seasons
masterTup = [('', 'PPR'),
             ('', 'Position'),
             ('', 'Team'),
             ('', 'Bye'),
             ('', 'Pts'),
             ('', 'Week'),
             ('', 'Year'),
             ('', 'Game'),
             ('', 'Opp'),
             ('Passing', 'Att'),
             ('Passing', 'Cmp'),
             ('Passing', 'Yds'),
             ('Passing', 'TD'),
             ('Passing', 'Int'),
             ('Passing', '2Pt'),
             ('Rushing', 'Att'),
             ('Rushing', 'Yds'),
             ('Rushing', 'TD'),
             ('Rushing', '2Pt'),
             ('Receiving', 'Rec'),
             ('Receiving', 'Yds'),
             ('Receiving', 'TD'),
             ('Receiving', '2Pt'),
             ('Fumbles', 'FL'),
             ('Fumbles', 'TD'),
             ('Defense', 'Sack'),
             ('Defense', 'Int'),
             ('Defense', 'Saf'),
             ('Defense', 'FR'), 
             ('Defense', 'Blk'),
             ('Defense', 'TD'),
             ('Defense', 'PA'),
             ('Defense', 'PassYds/G'),
             ('Defense', 'RushYds/G'),
             ('Defense', 'TotYds/G'),
             ('Kicking', 'XPA'),
             ('Kicking', 'XPM'),
             ('Kicking', 'FGA'),
             ('Kicking', 'FGM'),
             ('Kicking', '50+'),
            ]

In [425]:
#Set to change the index names 

indexDEF = ['Bye', 'Pts*', 'Sack', 'Int', 'Saf', 'FR', 'Blk', 'TD', 'PA',
       'PassYds/G', 'RushYds/G', 'TotYds/G']

indexKicker = ['Bye', 'Pts*', 'XPA', 'XPM', 'FGA', 'FGM', '50+']


In [426]:
#Load data from web for 2017 season
def loadWebData(frameArray):
    url_qb_season17 = 'https://www.footballdb.com/fantasy-football/index.html?pos=QB&yr=2017&wk=all&rules=1'
    url_rb_season17 = 'https://www.footballdb.com/fantasy-football/index.html?pos=RB&yr=2017&wk=all&rules=1'
    url_wr_season17 = 'https://www.footballdb.com/fantasy-football/index.html?pos=WR&yr=2017&wk=all&rules=1'
    url_te_season17 = 'https://www.footballdb.com/fantasy-football/index.html?pos=TE&yr=2017&wk=all&rules=1'
    url_def_season17 = 'https://www.footballdb.com/fantasy-football/index.html?pos=DST&yr=2017&wk=all&rules=1'
    url_kicker_season17 = 'https://www.footballdb.com/fantasy-football/index.html?pos=K&yr=2017&wk=all&rules=1'
    
    #Array [0] is where the data is stored. [1] stores the rules 
    frameArray.insert(QB, pd.io.html.read_html(url_qb_season17)[0])
    frameArray.insert(RB, pd.io.html.read_html(url_rb_season17)[0])
    frameArray.insert(WR, pd.io.html.read_html(url_wr_season17)[0])
    frameArray.insert(TE, pd.io.html.read_html(url_te_season17)[0])
    frameArray.insert(DST, pd.io.html.read_html(url_def_season17)[0]) 
    frameArray.insert(K, pd.io.html.read_html(url_kicker_season17)[0]) 

In [427]:
#Fix name that contains the next line string '\xa0'
#nameIndex: Empty index that will contain the fixed name
#data: Pandas Series type that contains the names of each player
def nameFix(nameIndex, data):
    #Get name from Player column and save to nameIndex
    NAME_FIX_OFFSET = 3
    index = nameIndex

    for i in data:
        index.append(i)


    size = len(index)

    for i in range(size):
        a1 = index[i]
        count = 0

        for x in a1:
            count += 1
            if x == u'\xa0':
                break

        count -= NAME_FIX_OFFSET
        index[i] = a1[:count]
    
    return index

In [428]:
def renameIndex(dframe, teamName):

    d1 = dict()
    index = 0

    for i in teamName:
        d1[index] = i
        index += 1
        
    dframe = (dframe.rename(d1, axis = 'index'))
    return(dframe)
    

In [429]:
def trimName(teamName):    
    arryIndex = len(teamName)
    
    
    for a1 in range(arryIndex):
        #a1 is index for the string array teamName 
        strLen = len(teamName.iloc[a1])
        for b1 in teamName.iloc[a1][::-1]:
            if b1.isupper():
                strLen -= 1
            else:
                break
                
        teamName.iat[a1] = (teamName.iloc[a1][:strLen])
       
    return teamName

In [430]:
def setDataFrame(dframe, dataType, index, fixXa0):
    nameIndex = []
    dataMatrix = []
    
    #Get the dimension of the frame. shape[0] will be the number of rows
    shape = dframe.shape
    
    #Go through each Series in the DataFrame from 0 to the number of rows
    #From each Series, extact the data and store it in a list matrix
    #Convert the list matrix to a np.array to store into DataFrame
    for count in range(shape[0]):
        a1 = []  
        temp = dframe.iloc[count].iloc[1:]    
        
        for i in temp:
            a1.append(i)

        dataMatrix.append(a1)
    
    dataMatrix = np.array(dataMatrix)
    
    if(fixXa0):
        nameIndex = nameFix(nameIndex, frameArray[dataType]['\xa0'].get("Player")) 
        dframe = DataFrame(dataMatrix, columns = index, index = nameIndex)
    else:
        dframe = DataFrame(dataMatrix, columns = index)
        
        #Create a dictionary of 'fixed' team names, then replace the original with the
        #fixed version
        if(dataType == DST):
            nameIndex = frameArray[dataType].get('Team')
            nameIndex = trimName(nameIndex)
        elif(dataType == K):
            nameIndex = nameFix(nameIndex, frameArray[dataType].get('Player'))               
                               
        d1 = dict()
        index = 0

        for i in nameIndex:
            d1[index] = i
            index += 1
        
        dframe = (dframe.rename(d1, axis = 'index'))
        dframe = dframe.rename({'Pts*': 'Pts'}, axis='columns')
        
        
        
    return(dframe)



In [675]:
def setDataFrame_weeks(dframe, keyList, index):
    nameIndex = []  #This is the name of the team or player
    dataMatrix = [] #This is the data going into dataframe
    position = keyList['Pos'] #Get the positionof the data
    dkeylist = {
            ('','Pos'): keyList['Pos'], 
            ('', 'PPR'): keyList['StdScore'],
            ('', 'Week'): keyList['Week'],
            ('', 'Year'): keyList['Year']
         }
    
    
    #Get the dimension of the frame. shape[0] will be the number of rows
    shape = dframe.shape
    
    #Go through each Series in the DataFrame from 0 to the number of rows
    #From each Series, extact the data and store it in a list matrix
    #Convert the list matrix to a np.array to store into DataFrame
    for count in range(shape[0]):
        a1 = []  
        temp = dframe.iloc[count].iloc[1:]    
        
        for i in temp:
            a1.append(i)

        dataMatrix.append(a1)
    
    dataMatrix = np.array(dataMatrix)
    
    #Fix the names for QB, WR, TE, RB
    if(position=='QB' or position=='RB' or position=='TE' or position=='WR' ):
        nameIndex = nameFix(nameIndex, dframe['\xa0'].get("Player")) 
        retframe = DataFrame(dataMatrix, columns = index, index = nameIndex)
    #Fix the names for Kicker and Defense
    elif(position=='DST' or position=='K'):
        dname = dict()
        count = 0
        retframe = DataFrame(dataMatrix, columns = index)
        
        #Create a dictionary of 'fixed' team names, then replace the original with the
        #fixed version
        if(position == 'DST'):
            nameIndex = dframe.get('Team')
            nameIndex = trimName(nameIndex)
        elif(position == 'K'):
            nameIndex = nameFix(nameIndex, dframe.get('Player'))               
                               
        for i in nameIndex:
            dname[count] = i
            count += 1
        #Rename the frame and change the Pts* to Pts
        retframe = (retframe.rename(dname, axis = 'index'))
        retframe = retframe.rename({'Pts*': 'Pts'}, axis='columns')
        
    #Create the dataframe for the keys using keyList argument and concat
    #with the returning frame
    kframe = DataFrame(dkeylist, nameIndex)
    retframe = pd.concat([retframe, kframe], axis=1, join_axes=[retframe.index])    
        
    return(retframe)


In [696]:
def createIndex(keyList):
    position = keyList['Pos'] 
    index = []
    
    if(position=='QB' or position=='RB' or position=='TE' or position=='WR' ):
        index = pd.MultiIndex.from_tuples(tupOffense)
    elif(position == 'K'):
        index = pd.MultiIndex.from_tuples(tupKicker)
    elif(position == 'DST'):
        index = pd.MultiIndex.from_tuples(tupDef)
        
    return index

In [670]:
#MAIN STARTS HERE
frameArray = []
loadWebData(frameArray)


In [783]:
#Create multi-index from tuple
index = pd.MultiIndex.from_tuples(tupOffense)
#frameQB = setDataFrame(frameArray[QB], QB, index, fixXa0 = True)
#frameRB = setDataFrame(frameArray[RB], RB, index, fixXa0 = True)
#frameWR = setDataFrame(frameArray[WR], WR, index, fixXa0 = True)
#frameTE = setDataFrame(frameArray[TE], TE, index, fixXa0 = True)

#index = pd.MultiIndex.from_tuples(tupDef)
#frameDef = setDataFrame(frameArray[DST], DST, index, fixXa0 = False)

#TODO - CREATE BOTTOM CODE WORKS. MAKE IT CLEAN! 
frameQB = setDataFrame_weeks(tframeArray[2][0], tframeArray[2][1] , index)

indexd = pd.MultiIndex.from_tuples(tupDef)
frameK  = setDataFrame_weeks(tframeArray[5][0], tframeArray[5][1], indexd)

#index = pd.MultiIndex.from_tuples(tupKicker)
#frameK = setDataFrame_weeks(tframeArray[4][0], tframeArray[4][1], index)


columns = pd.MultiIndex.from_tuples(masterTup)
frameMaster = DataFrame(columns = columns)

for tup in tframeArray:
    index = createIndex(tup[1])
    dframe = setDataFrame_weeks(tup[0], tup[1], index)
    frameArray.append(dframe)
    frameMaster = pd.concat([frameMaster, dframe])



In [785]:
frameMaster.loc['Peyton Manning']

...  \
                Bye     Game  Opp PPR Pos Position   Pts Team Week  Year ...   
Peyton Manning  NaN  IND@HOU  NaN   1  QB      NaN  35.0  NaN    1  2010 ...   
Peyton Manning  NaN  NYG@IND  NaN   1  QB      NaN  28.0  NaN    2  2010 ...   

               Passing      Receiving            Rushing             
                    TD  Yds       2Pt Rec TD Yds     2Pt Att TD Yds  
Peyton Manning       3  433         0   0  0   0       0   0  0   0  
Peyton Manning       3  255         0   0  0   0       0   3  0  -3  

[2 rows x 51 columns]

In [725]:
columns = pd.MultiIndex.from_tuples(masterTup)
frameMaster = DataFrame(columns = columns)

In [750]:
#frame = [frameMaster, frameKicker, frameQB]
#frameMaster = pd.concat([frameMaster, frameQB])
#frameMaster = pd.concat([frameMaster, frameQB], axis = 1, join_axes = [frameQB.index])


Passing  \
                     PPR Position Team  Bye  Pts Week Year Game  Opp     Att   
Hakeem Nicks         NaN      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN   
Chad Johnson         NaN      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN   
Austin Collie        NaN      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN   
Miles Austin         NaN      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN   
Wes Welker           NaN      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN   
Legedu Naanee        NaN      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN   
Reggie Wayne         NaN      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN   
Nate Washington      NaN      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN   
Greg Jennings        NaN      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN   
Steve Smith          NaN      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN   
Steve Breaston       NaN      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN   
Roddy White          NaN      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN   
Brandon Lloyd        NaN      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN   
Mark Clayton         NaN      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN   
Anquan Boldin        NaN      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN   
Hines Ward           NaN      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN   
Micheal Spurlock     NaN      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN   
Mohamed Massaquoi    NaN      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN   
Jeremy Maclin        NaN      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN   
Larry Fitzgerald     NaN      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN   
Mike Williams        NaN      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN   
Eddie Royal          NaN      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN   
Roscoe Parrish       NaN      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN   
Devery Henderson     NaN      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN   
Jabar Gaffney        NaN      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN   
Donald Driver        NaN      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN   
Kevin Walter         NaN      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN   
Mike Thomas          NaN      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN   
Jordan Shipley       NaN      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN   
Kassim Osgood        NaN      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN   
...                  ...      ...  ...  ...  ...  ...  ...  ...  ...     ...   
Derrick Mason        NaN      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN   
Brandon LaFell       NaN      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN   
Jacoby Jones         NaN      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN   
Bryant Johnson       NaN      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN   
T.J. Houshmandzadeh  NaN      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN   
Craig Davis          NaN      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN   
Joshua Cribbs        NaN      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN   
Greg Camarillo       NaN      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN   
Dominique Zeigler    NaN      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN   
Greg Lewis           NaN      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN   
Max Komar            NaN      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN   
James Jones          NaN      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN   
Darrius Heyward-Bey  NaN      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN   
Percy Harvin         NaN      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN   
Anthony Gonzalez     NaN      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN   
Ted Ginn Jr.         NaN      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN   
Yamon Figurs         NaN      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN   
Michael Crabtree     NaN      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN 

# TEST SECTION STARTS AFTER THIS LINE 

In [438]:
def setDataFrame_weeks_scratch(dframe, keyList, dataType, index, fixXa0):
    nameIndex = []  #This is the name of the team or player
    dataMatrix = [] #This is the data going into dataframe
    
    #Get the dimension of the frame. shape[0] will be the number of rows
    shape = dframe.shape
    
    #Go through each Series in the DataFrame from 0 to the number of rows
    #From each Series, extact the data and store it in a list matrix
    #Convert the list matrix to a np.array to store into DataFrame
    for count in range(shape[0]):
        a1 = []  
        temp = dframe.iloc[count].iloc[1:]    
        
        for i in temp:
            a1.append(i)

        dataMatrix.append(a1)
    
    dataMatrix = np.array(dataMatrix)
    
    
    #Fix the names for QB, WR, TE, RB
    if(fixXa0):
        nameIndex = nameFix(nameIndex, dframe['\xa0'].get("Player")) 
        dframe = DataFrame(dataMatrix, columns = index, index = nameIndex)
    #Fix the names for Kicker and Defense
    else:
        dframe = DataFrame(dataMatrix, columns = index)
        
        #Create a dictionary of 'fixed' team names, then replace the original with the
        #fixed version
        if(dataType == DST):
            nameIndex = frameArray[dataType].get('Team')
            nameIndex = trimName(nameIndex)
        elif(dataType == K):
            nameIndex = nameFix(nameIndex, frameArray[dataType].get('Player'))               
                               
        d1 = dict()
        index = 0

        for i in nameIndex:
            d1[index] = i
            index += 1
        
        dframe = (dframe.rename(d1, axis = 'index'))
        dframe = dframe.rename({'Pts*': 'Pts'}, axis='columns')
        
        
        
    #Create the dataframe for the keys using keyList argument 
    d2 = {('','Pos'): keyList['Pos'], 
          ('', 'PPR'): keyList['StdScore'],
          ('', 'Week'): keyList['Week'],
          ('', 'Year'): keyList['Year']
         }
    
    kframe = DataFrame(d2, nameIndex)
    dframe = pd.concat([dframe, kframe], axis=1, join_axes=[dframe.index])    
        

    return(dframe)


In [337]:
tupOffense = [('', 'Game'),        
              ('', 'Pts'),
              ('Passing', 'Att'),
              ('Passing', 'Cmp'),
              ('Passing', 'Yds'),
              ('Passing', 'TD'),
              ('Passing', 'Int'),
              ('Passing', '2Pt'),
              ('Rushing', 'Att'),
              ('Rushing', 'Yds'),
              ('Rushing', 'TD'),
              ('Rushing', '2Pt'),
              ('Receiving', 'Rec'),
              ('Receiving', 'Yds'),
              ('Receiving', 'TD'),
              ('Receiving', '2Pt'),
              ('Fumbles', 'FL'),
              ('Fumbles', 'TD')
             ]   

In [399]:
tframeArray = []
file = open('testpickle', 'rb')
tframeArray = pickle.load(file)
file.close()

In [501]:
tframeArray[6][0]
tframeArray[4][1]['Pos']

'K'

In [745]:
pd.concat([frameA, frameB], axis = 1, join_axes = [frameA.index])

blah                 new                 blah    
               Pos Week Score bye  yds  yds  yds  yds Score bye
Tom Brady       QB    2   100   2  100  100  100  100   100   2
Russel Wilson   QB    2   100   2  100  100  100  100   100   2

In [548]:
nameindex = ['Tom Brady', 'Russel Wilson']
t1 = {('blah', 'Pos'): 'QB', ('blah', 'Week'):2}
t2 = {('blah', 'Score'): 100, ('blah', 'bye'):2}
frameA = DataFrame(t1, index=nameindex)

In [549]:
frameA = DataFrame(t1, index=nameindex)
frameB = DataFrame(t2, index=nameindex)

In [554]:
frameA = pd.concat([frameA, frameB], axis = 1, join_axes = [frameA.index])

In [739]:
t3 = {('new', 'yds'): 100}
frameC = DataFrame(t3, index=nameindex)
frameA = pd.concat([frameA, frameC], axis = 1, join_axes = [frameA.index])

Index(['Tom Brady', 'Russel Wilson'], dtype='object')

In [301]:
{('blah', 'Pos'): tframeArray[0][1]['Pos']}

{('blah', 'Pos'): 'QB'}

In [609]:
tframeArray[4][0]

,Player,Game,Pts*,XPA,XPM,FGA,FGM,50+
0,John KasayJ. Kasay,CAR@NYG,12.0,1,1,3,3,1
1,Jeff ReedJ. Reed,ATL@PIT,11.0,0,0,5,3,1
2,Mason CrosbyM. Crosby,GB@PHI,11.0,3,3,2,2,1
3,Neil RackersN. Rackers,IND@HOU,10.0,4,4,2,2,0
4,Nick FolkN. Folk,BAL@NYJ,9.0,0,0,3,3,0
5,Matt BryantM. Bryant,ATL@PIT,9.0,0,0,4,3,0
6,Mike NugentM. Nugent,CIN@NE,8.0,3,3,1,1,1
7,Stephen GostkowskiS. Gostkowski,CIN@NE,8.0,5,5,3,1,0
8,Rob BironasR. Bironas,OAK@TEN,8.0,5,5,1,1,0
9,David AkersD. Akers,GB@PHI,8.0,2,2,2,2,0


In [616]:
tframeArray[4][1]

{'Pos': 'K', 'StdScore': 1, 'Week': 1, 'Year': 2010}